In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Import data

In [2]:
X_train = np.genfromtxt('sliced_data/X_train.csv', delimiter=',')
y_train = np.genfromtxt('sliced_data/y_train.csv', delimiter=',')
X_test = np.genfromtxt('sliced_data/X_test.csv', delimiter=',')
y_test = np.genfromtxt('sliced_data/y_test.csv', delimiter=',')
X_val = np.genfromtxt('sliced_data/X_val.csv', delimiter=',')
y_val = np.genfromtxt('sliced_data/y_val.csv', delimiter=',')

# Scale Data

## Min Max Scaler

In [3]:
scaler = MinMaxScaler()

i = 0
while i < X_train.shape[0]:
    X_train[i] = scaler.fit_transform(X_train[i].reshape(-1, 1)).reshape(40, )
    i += 1
    
i = 0
while i < X_test.shape[0]:
    X_test[i] = scaler.fit_transform(X_test[i].reshape(-1, 1)).reshape(40, )
    i += 1

# Standard Scaler

In [3]:
scaler = StandardScaler()

i = 0
while i < X_train.shape[0]:
    X_train[i] = scaler.fit_transform(X_train[i].reshape(-1, 1)).reshape(40, )
    i += 1
    
i = 0
while i < X_test.shape[0]:
    X_test[i] = scaler.fit_transform(X_test[i].reshape(-1, 1)).reshape(40, )
    i += 1

# Time Series Forest clf

In [11]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sklearn.metrics import accuracy_score
clf = TimeSeriesForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.5996152201795639

# Rocket clf

In [4]:
from sktime.classification.kernel_based import RocketClassifier
start = time.time()
clf = RocketClassifier()
clf.fit(X_train, y_train)
end = time.time()
print(end - start)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


1698.829479932785


In [5]:
y_pred = clf.predict(X_test)
print('test accuracy:', accuracy_score(y_test, y_pred))

val_pred = clf.predict(X_val)
print('val accuracy:', accuracy_score(y_val, val_pred))

print(classification_report(y_test, y_pred))

test accuracy: 0.7063916203505771
val accuracy: 0.7542450665442864
              precision    recall  f1-score   support

         0.0       0.63      1.00      0.77      9356
         1.0       1.00      0.41      0.58      9356

    accuracy                           0.71     18712
   macro avg       0.82      0.71      0.68     18712
weighted avg       0.82      0.71      0.68     18712



# Hive COTE clf

In [29]:
from sktime.classification.hybrid import HIVECOTEV2
from sktime._contrib.vector_classifiers._rotation_forest import RotationForest
hive_clf = HIVECOTEV2(
    stc_params={
        "estimator": RotationForest(n_estimators=3),
        "n_shapelet_samples": 100,
        "max_shapelets": 10,
        "batch_size": 20,
    },

    drcif_params={"n_estimators": 2, "n_intervals": 2, "att_subsample_size": 2},
    arsenal_params={"num_kernels": 50, "n_estimators": 3},
    tde_params={
        "n_parameter_samples": 10,
        "max_ensemble_size": 3,
        "randomly_selected_params": 5,
    },
)

In [ ]:
start = time.time()
hive_clf.fit(X_train, y_train)
end = time.time()
print(end - start)